# Прогнозирование заказов такси

## Задача

Компания «Чётенькое такси» собрала исторические данные о заказах такси в аэропортах. Чтобы привлекать больше водителей в период пиковой нагрузки, нужно спрогнозировать количество заказов такси на следующий час. Необходимо построить модель для такого предсказания.
Значение метрики **RMSE** на тестовой выборке должно быть **не больше 48**.

Условия:
- **ресемплирование** выполнить **по одному часу**,
- **тестовая выборка размером 10% от исходных данных**.

## Подключаемые библиотеки

In [ ]:
!pip install phik -q

In [ ]:
!pip install catboost -q

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import phik
import seaborn as sns

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler
)
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression
)
from sklearn.svm import SVR
from catboost import CatBoostRegressor

In [ ]:
RANDOM_STATE = 42

## Исходные данные

In [ ]:
try:
    taxi = pd.read_csv('taxi.csv', parse_dates=[0])
except:
    taxi = pd.read_csv('https://code.s3.yandex.net/datasets/taxi.csv', parse_dates=[0])

In [ ]:
taxi.head(5)

## Подготовка данных

In [ ]:
taxi.info()

In [ ]:
taxi.isna().sum()

In [ ]:
taxi.duplicated().sum()

In [ ]:
taxi = taxi.set_index('datetime')

In [ ]:
taxi = taxi.sort_index()

In [ ]:
taxi.info()

## Исследовательский анализ данных

### Признак "num_orders"

In [ ]:
taxi = taxi.resample('1H').sum()
taxi.head(5)

In [ ]:
taxi.describe()

В датафрейме присутствуют часы, за которые не было заказов

In [ ]:
taxi.hist()
plt.ylabel('Кол-во попаданий в диапазон, шт.')
plt.xlabel('Кол-во заказов в час, шт.')
plt.title('Гистограмма кол-ва заказов в час, шт.')
plt.show()

In [ ]:
taxi.boxplot()
plt.ylabel('Кол-во заказов в час, шт.')
plt.title('Диаграмма размаха')
plt.show()

В среднем за час поступает 84 заявки, выбросы, скорее всего, связаны с праздниками

Вычислим значение правого "уса"

In [ ]:
r_border = taxi.describe().loc['75%'] + (taxi.describe().loc['75%'] - taxi.describe().loc['25%']) * 1.5
r_border

In [ ]:
taxi.index.year.unique()

В датафрейме представлены данные только за 2018 год

Исследуем датафрейм на тренды и сезонность

In [ ]:
decomposed = seasonal_decompose(taxi)

In [ ]:
decomposed.seasonal.plot()
decomposed.trend.plot()
decomposed.resid.plot()
plt.legend(labels=['сезонность', 'тренды', 'шумы'])
plt.xlabel('Дата')
plt.ylabel('Кол-во заказов в час, шт.')
plt.title('Сезонность, тренды и шумы кол-ва заказов такси')
plt.show()

Есть явный тренд на увеличение кол-ва заказов такси. Сезонность тоже есть, но надо взять меньший временной интервал

In [ ]:
decomposed.seasonal.head(40).plot()
plt.xlabel('Дата')
plt.ylabel('Кол-во заказов в час, шт.')
plt.title('Сезонность кол-ва заказов такси (1 марта 2018 г. был четверг)')
plt.show()

Можно выявить следующую сезонность:
- с 18:00 до 24:00 люди спешат домой,
- до 06:00 тишина (все спят),
- после 06:00 все спешат на работу,
- пик в районе 16:00 скорее всего связан с забором детей из детских садов и школ.

### Признак "hours"

Добавим новый признак 'hours'

In [ ]:
taxi['hours'] = taxi.index.hour
taxi.head(5)

In [ ]:
taxi.groupby(by='hours').mean().plot(legend=False)
plt.xlabel('Час')
plt.ylabel('Среднее кол-во заказов за час, шт.')
plt.title('Среднее кол-во заказов за час, шт.')
plt.show()

Получили график сезонности, прослеживается явная связь между кол-вом заказов и часом

### Признак "months"

Добавим новый признак 'months'

In [ ]:
taxi['months'] = taxi.index.month
taxi.head(5)

In [ ]:
taxi.groupby(by='months').sum()['num_orders'].plot(legend=False)
plt.xlabel('Месяц')
plt.ylabel('Суммарное кол-во заказов в месяц, шт.')
plt.title('Суммарное кол-во заказов в месяц, шт.')
plt.show()

Видно, что при росте номера месяца растет и кол-во заказов

### Признак "dayofweek"

Добавим новый признак 'dayofweek'

In [ ]:
taxi['dayofweek'] = taxi.index.dayofweek
taxi.head(5)

In [ ]:
taxi.groupby(by='dayofweek').mean()['num_orders'].plot(legend=False)
plt.xlabel('День недели')
plt.ylabel('Среднее кол-во заказов, шт.')
plt.title('Среднее кол-во заказов в зависимости от дня недели, шт.')
plt.show()

Прослеживается зависимость между днем недели и кол-вом заказов:
- в понедельник (0) пик, так как многие опаздывают после выходных,
- в пятницу (4) пик, так как многие едут отдыхать,
- в воскресенье затишье, так как все сидят по домам

### Признак "shift_1"

Добавим новый признак 'shift_1'

In [ ]:
taxi['shift_1'] = taxi['num_orders'].shift()

In [ ]:
taxi.head(5)

### Признак "shift_2"

Добавим новый признак 'shift_2'

In [ ]:
taxi['shift_2'] = taxi['num_orders'].shift(2)

In [ ]:
taxi.head(5)

### Признак "shift_3"

Добавим новый признак 'shift_3'

In [ ]:
taxi['shift_3'] = taxi['num_orders'].shift(3)

In [ ]:
taxi.head(5)

### Признак "rolling_mean"

Добавим новый признак 'rolling_mean'

In [ ]:
taxi['rolling_mean'] = taxi['num_orders'].shift().rolling(8).mean()

In [ ]:
taxi.head(5)

### Анализ корреляции

In [ ]:
map = taxi.phik_matrix(interval_cols=['rolling_mean'])

In [ ]:
sns.heatmap(map, annot=True)
plt.title('Тепловая карта признаков датафрейма "taxi"')
plt.show()

## Построение модели

In [ ]:
train, test = train_test_split(taxi, shuffle=False, test_size=0.1, random_state=RANDOM_STATE)
train = train.dropna()
train.head(5)

In [ ]:
X_train = train.drop('num_orders', axis=1)
y_train = train['num_orders']

X_test = test.drop('num_orders', axis=1)
y_test = test['num_orders']

In [ ]:
num_columns = ['hours', 'months', 'dayofweek', 'shift_1', 'shift_2', 'shift_3', 'rolling_mean']

In [ ]:
data_preprocessor = ColumnTransformer(
    [
        ('num', StandardScaler(), num_columns)
    ], 
    remainder='passthrough'
)

In [ ]:
pipe_final = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models', DecisionTreeRegressor(random_state=RANDOM_STATE))
    ]
)

In [ ]:
param_grid = [
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'models__max_depth': range(1, 15),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },

    {
        'models': [LinearRegression()],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },

    {
        'models': [SVR()],
        'models__kernel': ['linear', 'rbf'],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },

    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__C': range(1, 5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },

    {
        'models': [CatBoostRegressor(random_state=RANDOM_STATE)],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    }
]

In [ ]:
grid_search = GridSearchCV(
    pipe_final,
    param_grid,
    n_jobs=-1,
    scoring='neg_root_mean_squared_error'
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
predicts = grid_search.predict(X_test)

In [ ]:
rmse = mean_squared_error(y_test, predicts, squared=False)

In [ ]:
print('Значение метрики RMSE на тестовой выборке', rmse)

## Вывод

Добиться нужного значения метрики удалось только при CatBoostRegressor

**Вопросы:**
1) почему модели очень чувствительны к значению у rolling. Например, при изменении окна с 7 на 8, метрика может измениться на 10 процентов. Можно ли считать такие колебания устойчивой моделью?